In [1]:
# qiskit circuit & machine
from qiskit import QuantumCircuit
import qiskit_aer
from qiskit_ibm_runtime.fake_provider import FakeVigo
from qiskit.circuit.library import QFT
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager

qft = QFT(5).to_gate()

qcb = QuantumCircuit(5,5)
qcb.h([0,1,2,3,4])
qcb.append(qft, [0, 1, 2, 3, 4])
qcb.measure_all()

optimization_level = 1
backend = qiskit_aer.AerSimulator.from_backend(FakeVigo())
pass_manager = generate_preset_pass_manager(backend=backend, optimization_level=optimization_level, seed_transpiler=1)

In [3]:
qcb_trans = pass_manager.run(qcb)
job = backend.run(qcb_trans, shots=10000)
count = job.result().get_counts()

In [4]:
esp = 1
for inst in qcb_trans.data:
    gate_error = 0
    if inst.operation.name not in ["barrier", "measure"]:
        gate_error = backend.properties().gate_error(inst.operation.name, [qubit._index for qubit in inst.qubits])
    elif inst.operation.name == "measure":
        gate_error = backend.properties().readout_error(inst.qubits[0]._index)
    esp = esp * (1 - gate_error)
esp

0.5748345441432775

In [5]:
import numpy as np
def alt_output(value, prob, states):
    alts = states.copy()
    alts.remove(value)
    rng = np.random.default_rng()
    true_obs = rng.binomial(1, prob, 1)
    if true_obs == 1:
        return value
    else:
        return np.random.choice(alts, 1)[0]

In [6]:
org_sample = []
for key in count:
    for i in range(count[key]):
        org_sample.append(key)

resamples = []
sample_times = 100
states = list(count.keys())
for i in range(sample_times):
    sample = [alt_output(val, esp, states) for val in org_sample]
    resamples.append(sample)

In [38]:
np.sum(np.repeat(list(count.keys()), list(count.values())) == np.array(org_sample))

10000

In [7]:
resample_counts = []
for sample in resamples:
    values, counts = np.unique(sample, return_counts=True)
    count_dict = {}
    for i in range(len(values)):
        count_dict[values[i]] = counts[i]
    resample_counts.append(count_dict)

In [20]:
resample_stats = {}
resample_count_values = {}
ci_alpha = 0.95
def get_ci_range(alp):
    alp_bor = (1 - alp) * 100 / 2
    return [alp_bor, 100-alp_bor]
ci_range = get_ci_range(ci_alpha)
# todo: study about bootstrap confidence interval
for state in states:
    count_values = [sample[state] for sample in resample_counts]
    mean = np.mean(count_values)
    std = np.std(count_values)
    interval = np.percentile(count_values, ci_range)
    resample_count_values[state] = count_values
    resample_stats[state] = {
        "state": state,
        "mean": mean,
        "std": std,
        "interval": interval,
        "lb": interval[0],
        "ub": interval[1]
    }
resample_stats

{'10011 00000': {'state': '10011 00000',
  'mean': 139.4,
  'std': 11.563736420379012,
  'interval': array([116.9  , 157.525]),
  'lb': 116.9,
  'ub': 157.52499999999998},
 '10111 00000': {'state': '10111 00000',
  'mean': 141.54,
  'std': 11.411765858095757,
  'interval': array([117.   , 160.525]),
  'lb': 117.0,
  'ub': 160.52499999999998},
 '11001 00000': {'state': '11001 00000',
  'mean': 145.16,
  'std': 11.80654056021492,
  'interval': array([122.375, 166.   ]),
  'lb': 122.37500000000001,
  'ub': 166.0},
 '01101 00000': {'state': '01101 00000',
  'mean': 137.22,
  'std': 12.516053691160005,
  'interval': array([113.475, 159.575]),
  'lb': 113.47500000000001,
  'ub': 159.575},
 '01001 00000': {'state': '01001 00000',
  'mean': 148.29,
  'std': 12.558897244583221,
  'interval': array([125.95 , 171.525]),
  'lb': 125.95,
  'ub': 171.52499999999998},
 '10001 00000': {'state': '10001 00000',
  'mean': 165.57,
  'std': 10.991137338783462,
  'interval': array([144.95, 188.  ]),
  'lb':

In [21]:
import pandas as pd
resample_pd = pd.DataFrame(resample_stats).T

In [33]:
import altair as alt

points = alt.Chart(resample_pd).mark_bar(color="#ffc400").encode(
  y=alt.Y('mean'),
  x=alt.X('state')
)

error_bars = alt.Chart(resample_pd).mark_rule().encode(
  alt.Y('lb').scale(zero=False),
  alt.Y2('ub'),
  alt.X('state')
)

points + error_bars


alt.LayerChart(...)